# E-76 Obligated Federal Funds Data Exploration
### From the Division of Local Assistance 

E-76's or "Authorizations to Proceed" is a federal authorization of funds to establish the reimbursement date for a phase of work 

#### Data Sources: 
- https://dot.ca.gov/programs/local-assistance/reports/e-76-obligated

In this notebook, we will be looking to see what types of data is in the spreadsheet as well as begin to clean it for futher analysis


In [5]:
##one time load 
#!pip install openpyxl

In [6]:
import pandas as pd
from siuba import _, mutate, count, filter, group_by, ungroup
import numpy as np

## Reading Excel as a data dictionary
Can use this approach, but recommend the next with second one. 

In [9]:
#obligated = pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/obligated-projects.xlsx', sheet_name=['2014-2018', '2019', '2020', '2021'])


In [3]:
#print(obligated)

In [4]:
#print(type(obligated))

In [5]:
#len(obligated)

In [6]:
#print(type(obligated['2020']))

In [7]:
#obligated['2020'].info()

In [8]:
#obligated['2021'].info()

In [9]:
#obligated['2019'].info()

In [10]:
#obligated['2014-2018'].info()

In [11]:
#df1 = obligated['2014-2018']
#df2 = obligated['2019']
#df3 = obligated['2020']
#df4 = obligated['2021']

In [12]:
#pd_dct = {"pre2018": df1, "2019": df2, "2020":df3, "2021":df4}

In [13]:
#mdf = pd.concat(pd_dct, axis=0).reset_index(level=0).rename({'level_0':'key'}, axis=1)

In [14]:
#mdf

## Reading Excel with Concat

In [7]:
#attempt 2
## code source: https://pbpython.com/pandas-excel-tabs.html
df = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/obligated-projects.xlsx', sheet_name=None), ignore_index=True)


In [8]:
df.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,...,9.0,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,...,7.0,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,...,6.0,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,...,0.0,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,...,0.0,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,NaN


In [10]:
df.tail()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
20016,Obligated,BRLO,5955(095),Orange County,2021-08-20 00:00:00,2021-08-27 00:00:00,2021-08-31 00:00:00,2021-09-01 00:00:00,2021-09-17 00:00:00,82000.0,...,16.0,ORA020501,"Silverado Canyon Road Over Silverado Creek, Ap...",Bridge Replacement (tc).,2,2021-08-20 00:00:00,2021-08-27 00:00:00,SCAG,NaT,NaN
20017,Obligated,HSIPL,5073(093),Orange,2021-08-17 00:00:00,2021-08-27 00:00:00,2021-08-30 00:00:00,2021-08-30 00:00:00,2021-09-08 00:00:00,21000.0,...,9.0,SCAG015,Intersection Of Glassell St. & Palmyra Ave.,Install A New 2-phase Traffic Signal,2,2021-07-28 00:00:00,2021-08-26 00:00:00,SCAG,NaT,NaN
20018,Obligated,HSIPL,5073(078),Orange,2021-08-18 00:00:00,2021-08-27 00:00:00,2021-09-01 00:00:00,2021-09-01 00:00:00,2021-09-17 00:00:00,0.0,...,16.0,SCAG015,Intersection Of Glassell Street And Walnut Avenue,Protected Left-turn Signal Phasing,4,2021-08-13 00:00:00,2021-08-27 00:00:00,SCAG,NaT,NaN
20019,Obligated,HSIPL,5073(080),Orange,2021-08-18 00:00:00,2021-08-27 00:00:00,2021-09-01 00:00:00,2021-09-01 00:00:00,2021-09-17 00:00:00,0.0,...,16.0,SCAG015,Intersection Of Chapman Avenue And James Street,Install Protected Left Turn Signal Phasing,4,2021-08-12 00:00:00,2021-08-27 00:00:00,SCAG,NaT,NaN
20020,Obligated,CMSTPLN,6071(129),Orange County Transportation Authority,2021-08-23 00:00:00,2021-08-24 00:00:00,2021-08-31 00:00:00,2021-08-31 00:00:00,2021-09-09 00:00:00,8000000.0,...,9.0,ORA100511,State Route 55 From Interstate 405 To Intersta...,Add One Mixed-flow And High-occupancy Lane In ...,6,2021-06-01 00:00:00,2021-08-22 00:00:00,SCAG,NaT,NaN


In [11]:
print(len(df))

20021


In [12]:
df.sample(10)

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,...,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
1879,Obligated,STPL,5929(293),San Joaquin County,2018-04-05,2018-04-05,2018-04-16,2018-04-16,2018-04-19,73842.00,...,3.0,SJ07-3071,"1)eight Mile Road: Interstate 5 To 3,060¿ East...",Rubberized Chip Seal/roadway Resurfacing Vario...,3,2018-04-02,2018-04-02,SJCG,NaN,NaN
7112,Obligated,CML,5392(056),Thousand Oaks,2016-03-16,2016-03-16,2016-03-21,2016-03-24,2016-04-01,87480.00,...,8.0,VEN150622,Moorpark Road North From Olsen Road To Crossri...,"Placement Of Concrete Sidewalks Curbs, Curb Ra...",1,NaN,NaN,SCAG,NaN,NaN
848,Obligated,DEM08L,5006(719),Los Angeles,2018-07-02,2018-07-11,2018-07-18,2018-07-26,2018-08-07,0.00,...,12.0,LA0G686,Highland Park Along Figueroa Between Ave 50 An...,Pedestrian Improvements,3,2018-06-28,2018-07-02,SCAG,NaN,NaN
18482,Obligated,CML,5063(181),Santa Ana,2021-02-02 00:00:00,2021-02-02 00:00:00,2021-02-04 00:00:00,2021-02-05 00:00:00,2021-02-08 00:00:00,937200.00,...,3.0,ORA152213,Throughout The City Of Santa Ana And At The Sa...,Install Bicycle Racks And Construct Bicycle Ce...,2,2021-02-01 00:00:00,2021-02-02 00:00:00,SCAG,NaT,NaN
3654,Obligated,ATPL,5378(038),Palmdale,2017-07-25,2017-08-07,2017-08-14,2017-08-17,2017-08-24,2500000.00,...,7.0,LASRTS,Ave R Between Sierra Highway And 25th Street.,Ave R Complete Street With Sidewalks Gap Closu...,2,2017-06-01,2017-08-07,SCAG,NaN,NaN
11964,Obligated,STPL,5301(012),Orange Cove,2014-04-28,2014-04-29,2014-04-30,2014-05-05,2014-05-05,0.00,...,0.0,FRE110630,Center Street From South Ave. To Park Blvd.,Reconstruction And Resurfacing(tc),3,NaN,NaN,CFCG,NaN,NaN
1800,Obligated,HSIPL,5933(097),Alameda County,2018-04-26,2018-04-26,2018-04-26,2018-04-26,2018-04-27,-7034.75,...,1.0,VAR110007,Multiple Locations - See Scomment For Details,Repair / Replace Drainage Inlet,4,NaN,NaN,MTC,NaN,NaN
19563,Obligated,HSIPL,5006(793),Los Angeles,2021-06-10 00:00:00,2021-06-30 00:00:00,2021-07-02 00:00:00,2021-07-02 00:00:00,2021-07-14 00:00:00,0.00,...,12.0,SCAG015,"W 3rd St/hoover St, W Anaheim St/bay View Ave,...",Install New Traffic Signals At All 4 Locations,3,2020-12-30 00:00:00,2021-06-29 00:00:00,SCAG,NaT,NaN
9537,Obligated,HSIPLN,5466(017),Yucca Valley,2015-03-25,2015-03-25,2015-03-25,2015-03-25,2015-03-29,0.00,...,4.0,SCAG015,Twentynine Palms Highway (sr 62) From La Honda...,"Raised Median Islands, Curb, Gutter, And Sidewalk",2,NaN,NaN,SCAG,NaN,NaN
8293,Obligated,CMLNI,5941(107),Madera County,2015-08-27,2015-08-28,2015-08-31,2015-09-04,2015-09-10,22000.00,...,6.0,MAD102071,County Of Madera,Vehicle Purchase,1,NaN,NaN,MCTC,NaN,NaN


## Data Cleaning

In [13]:
df.Agency.unique()

array(['Humboldt County', 'Mendocino County', 'Sacramento County',
       'Sutter County', 'Contra Costa County', 'Dinuba', 'Arvin',
       'Fresno', 'Tulare County', 'Palmdale', 'Los Angeles County',
       'Orange County Transportation Authority', 'Yolo County',
       'Nevada County', 'Fresno County', 'Ridgecrest', 'Lassen County',
       'Modoc County', 'Butte County', 'Yuba City', 'Concord',
       'Napa County', 'Pismo Beach', 'Ojai', 'Los Angeles', 'Chino Hills',
       'Indio', 'Moreno Valley', 'Turlock', 'Riverbank', 'Stockton',
       'Chula Vista', 'Sonoma County', 'Santa Clara County',
       'Kings County', 'Long Beach', 'Norwalk', 'Rancho Palos Verdes',
       'Southern California Association of Governments', 'Pasadena',
       'Riverside', 'Carlsbad', 'Santa Barbara County', 'Delano',
       'Merced County', 'South Lake Tahoe', 'Yuba County',
       'Metropolitan Transportation Commission', 'San Mateo',
       'Solano Transportation Authority', 'University Of California'

Some Agencies are all caps, others are not. Applying str.title() to correct this. 

In [14]:
df.Agency = df.Agency.str.title()

In [15]:
df.Agency.unique()

array(['Humboldt County', 'Mendocino County', 'Sacramento County',
       'Sutter County', 'Contra Costa County', 'Dinuba', 'Arvin',
       'Fresno', 'Tulare County', 'Palmdale', 'Los Angeles County',
       'Orange County Transportation Authority', 'Yolo County',
       'Nevada County', 'Fresno County', 'Ridgecrest', 'Lassen County',
       'Modoc County', 'Butte County', 'Yuba City', 'Concord',
       'Napa County', 'Pismo Beach', 'Ojai', 'Los Angeles', 'Chino Hills',
       'Indio', 'Moreno Valley', 'Turlock', 'Riverbank', 'Stockton',
       'Chula Vista', 'Sonoma County', 'Santa Clara County',
       'Kings County', 'Long Beach', 'Norwalk', 'Rancho Palos Verdes',
       'Southern California Association Of Governments', 'Pasadena',
       'Riverside', 'Carlsbad', 'Santa Barbara County', 'Delano',
       'Merced County', 'South Lake Tahoe', 'Yuba County',
       'Metropolitan Transportation Commission', 'San Mateo',
       'Solano Transportation Authority', 'University Of California'

In [16]:
pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.max_columns', None)

In [17]:
df.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,Waiting Days,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,NaN,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,NaN,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,NaN,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,NaN,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,NaN


In [18]:
df['Waiting Days'].unique()

array([nan])

In [19]:
df.drop('Waiting Days', axis=1, inplace=True)

In [20]:
df['Today'].unique()

array([nan, '2020-01-01', '2019-06-06 9:00 am', '2021-01-04',
       Timestamp('2021-10-01 00:00:00'), NaT], dtype=object)

In [21]:
df.drop('Today', axis=1, inplace=True)

In [22]:
df.drop('Unnamed: 28', axis=1, inplace=True)

In [23]:
df.rename(columns={'Project NO': 'project_no',
                        'Fed requested': 'fed_requested',
                        'AC requested': 'ac_requested',
                        'total requested': 'total_requested',
                        'FTIP No': 'ftip_no',
                        'project location': 'project_location'}, inplace=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20021 entries, 0 to 20020
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Location                20021 non-null  object 
 1   PREFIX                  20021 non-null  object 
 2   project_no              20021 non-null  object 
 3   Agency                  20021 non-null  object 
 4   prepared date           20011 non-null  object 
 5   submit  to HQ date      20011 non-null  object 
 6   HQ review date          20008 non-null  object 
 7   submit to FHWA date     20012 non-null  object 
 8   To FMIS date            20012 non-null  object 
 9   fed_requested           20020 non-null  float64
 10  ac_requested            20021 non-null  float64
 11  total_requested         20020 non-null  object 
 12  Status Comment          20021 non-null  object 
 13  Locode                  18522 non-null  object 
 14  DIST                    20021 non-null

In [25]:
df['total_requested'] = df['total_requested'].astype(float)

ValueError: could not convert string to float: '2748.3NA999'

In [26]:
df.loc[df['total_requested']=='2748.3NA999']

,Location,PREFIX,project_no,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,fed_requested,ac_requested,total_requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,ftip_no,project_location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO
15547,Obligated,ER,38Y0(001),Los Angeles County,2019-11-12 00:00:00,2019-12-04 00:00:00,2019-12-05 00:00:00,2019-12-06 00:00:00,2019-12-19 00:00:00,1017276.52,-1019159.54,2748.3NA999,Authorized,5953,7,E-76 approved on,28.00,2.00,13.00,NaN,"Mulholland Hwy, Et Al Guardrail Replacement","Install Temporary Traffic Signs, Remove And Re...",2,2019-11-06 00:00:00,2019-11-13 00:00:00,SCAG


**Issue here**, and in the following row where 'NA' appears between numbers. Need to determine whether or not to manually override in the exel sheet. 

In [30]:
print(df['prepared date'].dtypes)

datetime64[ns]


In [31]:
df['prepared date'].unique()

array(['2018-12-18T00:00:00.000000000', '2018-12-17T00:00:00.000000000',
       '2018-12-07T00:00:00.000000000', ...,
       '2021-09-02T00:00:00.000000000', '2021-09-01T00:00:00.000000000',
       '2021-09-08T00:00:00.000000000'], dtype='datetime64[ns]')

In [32]:
cols = ['prepared date','To FMIS date','submit to FHWA date','submit  to HQ date','HQ review date','Date Request Initiated','date completed request']
df[cols] = df[cols].applymap(lambda x : pd.to_datetime(x, format = '%Y-%m-%d'))


In [33]:
df.head()

,Location,PREFIX,project_no,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,fed_requested,ac_requested,total_requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,ftip_no,project_location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20021 entries, 0 to 20020
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Location                20021 non-null  object        
 1   PREFIX                  20021 non-null  object        
 2   project_no              20021 non-null  object        
 3   Agency                  20021 non-null  object        
 4   prepared date           20011 non-null  datetime64[ns]
 5   submit  to HQ date      20011 non-null  datetime64[ns]
 6   HQ review date          20008 non-null  datetime64[ns]
 7   submit to FHWA date     20012 non-null  datetime64[ns]
 8   To FMIS date            20012 non-null  datetime64[ns]
 9   fed_requested           20020 non-null  float64       
 10  ac_requested            20021 non-null  float64       
 11  total_requested         20020 non-null  object        
 12  Status Comment          20021 non-null  object

In [35]:
#compare_col2 = np.where(df["Date Request Initiated"] == df["date completed request"], True, False)
#df["compare_start_end"] = compare_col2
#df.head()

In [36]:
df[["ProjectID", "ProjectNO"]] = df["project_no"].str.split(pat="(", expand=True)


In [37]:
df.ProjectNO = [x.replace(")", "") for x in df.ProjectNO]

In [38]:
df.head()

,Location,PREFIX,project_no,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,fed_requested,ac_requested,total_requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,ftip_no,project_location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,5904,121
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,32D0,008
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,4820,004
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,5924,244
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,5924,214


## Locode and Project ID Columns 

Hypothesis:
    Project IDs and Locodes, or unique agency identifiers, are the same 
    
As it stands now, these columns have a mix of letters and numbers, making it hard to compare the two. 
In this block we will work to convert these columns, while trying to not convert the non-numeric numbers. 

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20021 entries, 0 to 20020
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Location                20021 non-null  object        
 1   PREFIX                  20021 non-null  object        
 2   project_no              20021 non-null  object        
 3   Agency                  20021 non-null  object        
 4   prepared date           20011 non-null  datetime64[ns]
 5   submit  to HQ date      20011 non-null  datetime64[ns]
 6   HQ review date          20008 non-null  datetime64[ns]
 7   submit to FHWA date     20012 non-null  datetime64[ns]
 8   To FMIS date            20012 non-null  datetime64[ns]
 9   fed_requested           20020 non-null  float64       
 10  ac_requested            20021 non-null  float64       
 11  total_requested         20020 non-null  object        
 12  Status Comment          20021 non-null  object

In [40]:
df.sample(10)

,Location,PREFIX,project_no,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,fed_requested,ac_requested,total_requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,ftip_no,project_location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO
9922,Obligated,RPSTPL,5030(052),Vallejo,2015-02-05,2015-02-05,2015-02-06,2015-02-06,2015-02-10,-231743.69,0.00,-443201.44,Authorized,5030.00,4,E-76 approved on E-76 approved on,0.00,1.00,4.00,SOL050048,"Sacramento St. From Georgia To Maine, Virginia...",Downtown Streetscape,2,NaT,NaT,MTC,5030,052
5873,Obligated,CML,5240(025),Culver City,2016-06-30,2016-07-05,2016-07-19,2016-08-24,2016-09-08,0.00,0.00,0.00,Authorized,5240.00,7,E-76 approved on Prepared on 9/29/2016,5.00,50.00,15.00,LAF1717,Down Town Area Of City Of Culver City,Realtime Parking Monitoring Project,5,NaT,NaT,SCAG,5240,025
7456,Obligated,DEM06,6207(062),Caltrans,2015-11-16,2016-01-19,2016-01-29,2016-02-11,2016-02-12,3073000.00,97006000.00,201391000.00,Authorized,6207.00,7,E-76 approved on Prepared on 10/6/2015,64.00,23.00,1.00,LA0D73,I-5 Carpool Lane-orange County Line To 605 (se...,Roadway Widening (hov)(tc),6,NaT,NaT,NON-MPO,6207,062
9216,Obligated,BRLS,5942(245),Fresno County,2015-03-20,2015-04-23,2015-05-14,2015-05-15,2015-05-20,354120.00,0.00,400000.00,Authorized,5942.00,6,E-76 approved on E-76 approved on,34.00,22.00,5.00,FRE040501,Burrough Valley Road Over Dry Creek (42c0134),Replace Timber 2 Lane Bridge With 2 Lane Bridge,2,NaT,NaT,CFCG,5942,245
10291,Obligated,HRRRL,5929(253),San Joaquin,2014-12-18,2014-12-18,2014-12-22,2014-12-23,2014-12-24,703800.00,0.00,911000.00,Authorized,NaN,10,E-76 approved on E-76 approved on,0.00,5.00,1.00,SJ07-3116,Peltier Rd From 300' West Bender Rd. To Lower ...,Widen & Improve Shoulders And Striping,3,NaT,NaT,SCAG,5929,253
16821,Obligated,ER,32L0(409),Plumas County,2020-06-30,2020-06-30,2020-06-30,2020-07-10,2020-07-13,437159.00,-437159.00,0.00,Authorized,5909,2,E-76 approved on,0.00,10.00,3.00,NaN,"Cr 511, Quincy Laporte Road At Willow Creek Fr...",(eo) Repair Shoulders And Vertical Drop-offs I...,4,2020-06-30,2020-06-30,NON-MPO,32L0,409
3276,Obligated,HSIPL,5487(002),Jurupa Valley,2017-09-28,2017-10-02,2017-10-10,2017-10-13,2017-10-20,135000.00,0.00,150000.00,Authorized,5487.00,8,E-76 approved on,34.00,11.00,7.00,SCAG015,Intersections In A 1.41 Mile Segment Of Pedley...,Add Left-turn Lanes; Modify Storm Drain Under-...,1,2017-08-28,2017-08-29,SCAG,5487,002
11146,Obligated,STPL,5173(016),Fowler,2014-08-04,2014-08-04,2014-08-04,2014-08-04,2014-08-04,0.00,0.00,0.00,Authorized,5173.00,6,E-76 approved on E-76 approved on,0.00,0.00,0.00,FRE 020610,On Fowler Ave From Merced St To South Ave,"Ac Overlay, Curb, Gutter & Sidewalks (tc)",5,NaT,NaT,CFCG,5173,016
12011,Obligated,CML,5063(149),Santa Ana,2014-04-10,2014-04-24,2014-04-25,2014-04-25,2014-04-29,229754.00,0.00,266822.00,Authorized,NaN,12,E-76 approved on E-76 approved on,14.00,1.00,4.00,ORA113020,"Maple Street Bike Trail At Occidental St., St....",Bulb Outs At Existing Curbs,2,NaT,NaT,NON-MPO,5063,149
11026,Obligated,NHPI,6052(967),Sunline Transit Agency,2014-07-28,2014-07-28,2014-07-28,2014-08-07,2014-08-15,31129927.00,-28694350.00,2455977.00,Authorized,6052.00,7,E-76 approved on E-76 approved on,0.00,10.00,8.00,LALS02,"In Long Beach,lakewood,cerritos Norwalk,santa ...",Pavement Preservation (tc),2,NaT,NaT,NON-MPO,6052,967


In [41]:
## dataset has a column NBIL, 32L0, 40A0, 5.92E+03, 38Y0, 38R0

## for now, leaving the errors. 

In [42]:
def get_num(x):
    try:
        return int(x)
    except Exception:
        try:
            return float(x)
        except Exception:
            return x

        
# https://stackoverflow.com/questions/56474599/how-to-ignore-errors-in-pandas-to-numeric-and-change-str-to-int

In [43]:
df['Locode'] = df['Locode'].apply(get_num)

In [44]:
df['ProjectID'] = df['ProjectID'].apply(get_num)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20021 entries, 0 to 20020
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Location                20021 non-null  object        
 1   PREFIX                  20021 non-null  object        
 2   project_no              20021 non-null  object        
 3   Agency                  20021 non-null  object        
 4   prepared date           20011 non-null  datetime64[ns]
 5   submit  to HQ date      20011 non-null  datetime64[ns]
 6   HQ review date          20008 non-null  datetime64[ns]
 7   submit to FHWA date     20012 non-null  datetime64[ns]
 8   To FMIS date            20012 non-null  datetime64[ns]
 9   fed_requested           20020 non-null  float64       
 10  ac_requested            20021 non-null  float64       
 11  total_requested         20020 non-null  object        
 12  Status Comment          20021 non-null  object

In [46]:
compare_col = np.where(df["Locode"] == df["ProjectID"], True, False)
df["compare_id_locode"] = compare_col
df.head()

,Location,PREFIX,project_no,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,fed_requested,ac_requested,total_requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,ftip_no,project_location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO,compare_id_locode
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,5904,121,True
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,32D0,008,False
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,4820,004,False
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,5924,244,True
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,5924,214,True


In [47]:
df['compare_id_locode'].value_counts()

True     16316
False     3705
Name: compare_id_locode, dtype: int64

In [48]:
group = df.groupby('Agency')
crosswalk_all = group.apply(lambda x: x['Locode'].unique())
print(crosswalk_all)

Agency
Access Services                                               [6312]
Agoura Hills                                             [5435, nan]
Alameda                                                       [5014]
Alameda - Contra Costa Transit District                       [6002]
Alameda Corridor Transportation Authority                     [6246]
                                                      ...           
Yrts                                                           [nan]
Yuba City                                                     [5163]
Yuba County                                                   [5916]
Yucaipa                                      [5457, nan, 5954, NBIL]
Yucca Valley                                             [5466, nan]
Length: 667, dtype: object


In [49]:
crosswalk_all.to_csv('crosswalk_all.csv')

In [50]:
#https://stackoverflow.com/questions/29530232/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe

nan_rows = df[df['Locode'].isnull()]

In [51]:
nan_rows.sample(10)

,Location,PREFIX,project_no,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,fed_requested,ac_requested,total_requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,ftip_no,project_location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO,compare_id_locode
12979,Obligated,CML,5958(059),Imperial,2014-01-07,2014-01-07,2014-01-07,2014-01-07,2014-01-07,-0.01,0.00,5631.38,Authorized,NaN,11,E-76 approved on E-76 approved on,0.00,0.00,0.00,NaN,Imperial County,Vehicle Purchase,4,NaT,NaT,SCAG,5958,059,False
10127,Obligated,STPLN,6071(096),Ora Co Trans Au,2014-11-24,2015-01-08,2015-01-09,2015-01-09,2015-01-14,7000000.00,0.00,9000000.00,Authorized,NaN,12,E-76 approved on E-76 approved on,45.00,1.00,5.00,ORA150110,State Route 91 From State Route 57 To State Ro...,Add Mixed-flow Lane In Each Direction,1,NaT,NaT,SCAG,6071,096,False
10004,Obligated,BPMPL,5181(172),Huntington Beac,2013-07-25,2015-01-08,2015-01-21,2015-01-21,2015-01-28,61971.00,0.00,70000.00,Authorized,NaN,12,E-76 approved on E-76 approved on,532.00,13.00,7.00,ORA020501,"Brookhurst Street Over Talbert Channel, Br. No...",Bridge Preventive Maintenance,3,NaT,NaT,SCAG,5181,172,False
10295,Obligated,STPL,5004(135),San Diego,2014-12-19,2014-12-19,2014-12-19,2014-12-19,2014-12-23,-253182.36,0.00,0.00,Authorized,NaN,11,E-76 approved on E-76 approved on,0.00,0.00,4.00,SD83,Friars Road And Sr-163,Road And Bridge Widening,3,NaT,NaT,SCAG,5004,135,False
11416,Obligated,CML,5010(006),Sonora,2014-06-18,2014-06-24,2014-07-09,2014-07-09,2014-07-10,243000.00,0.00,320000.00,Authorized,NaN,10,E-76 approved on E-76 approved on,6.00,15.00,1.00,TRANS-HUB,S. Washington St. And Stockton St/hwy 49 (see ...,"Const Ped,transit Facility And Traffic Improv",1,NaT,NaT,STANCOG,5010,006,False
9161,Obligated,BHLS,5282(040),Palm Springs,2015-05-01,2015-05-07,2015-05-23,2015-05-23,2015-05-27,1252434.00,0.00,1718000.00,Authorized,NaN,8,E-76 approved on E-76 approved on,6.00,16.00,4.00,RIV110124,Ramon Road From San Luis Rey Drive To Landau B...,Rehabilitate And Widen Bridge And Roadway,2,NaT,NaT,SCAG,5282,040,False
9278,Obligated,HSIPL,5450(081),Santa Clarita,2015-05-06,2015-05-06,2015-05-07,2015-05-08,2015-05-09,0.00,0.00,-6516.00,Authorized,NaN,7,E-76 approved on E-76 approved on,0.00,2.00,1.00,SCAG015,"Newhall Ave At Intersection Of 8th St. East , ...",Intersection Improvements.,3,NaT,NaT,SCAG,5450,081,False
8967,Obligated,CML,5169(040),El Centro,2015-06-12,2015-06-12,2015-06-15,2015-06-15,2015-06-16,442149.00,0.00,719556.00,Authorized,NaN,11,E-76 approved on E-76 approved on,0.00,3.00,1.00,IMP120619,Buena Vista Between 6th/8th St.,"Construct Sidewalk, C&g,ada Ramps",2,NaT,NaT,SCAG,5169,040,False
9226,Obligated,BHLS,5192(020),Tracy,2015-04-24,2015-05-01,2015-05-12,2015-05-14,2015-05-18,79677.00,-4671201.00,-5186404.00,Authorized,NaN,10,E-76 approved on E-76 approved on,7.00,13.00,4.00,SJ07-3002,11th Street - East Tracy Overhead (bridge 29c0...,Bridge Replacement - Seismic,10,NaT,NaT,SCAG,5192,020,False
12189,Obligated,STPL,5165(067),Turlock,2014-03-27,2014-03-27,2014-04-09,2014-04-11,2014-04-11,0.00,0.00,-121748.00,Authorized,NaN,10,E-76 approved on E-76 approved on,0.00,15.00,0.00,VA02,"Various Locations; E. Main St., Colorado Ave.,...",Roadway Rehabilitation And Resurfacing,3,NaT,NaT,MCAG,5165,067,False


In [52]:
print(len(nan_rows))

1499


In [53]:
#checking to see if these NaN values are also in the downloadable data. 

In [54]:
# There are the same number of NaNs (1499) on the excel sheet. 
# So without these, there are still 2,206 

## Comparing columns and Summarizing Data

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20021 entries, 0 to 20020
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Location                20021 non-null  object        
 1   PREFIX                  20021 non-null  object        
 2   project_no              20021 non-null  object        
 3   Agency                  20021 non-null  object        
 4   prepared date           20011 non-null  datetime64[ns]
 5   submit  to HQ date      20011 non-null  datetime64[ns]
 6   HQ review date          20008 non-null  datetime64[ns]
 7   submit to FHWA date     20012 non-null  datetime64[ns]
 8   To FMIS date            20012 non-null  datetime64[ns]
 9   fed_requested           20020 non-null  float64       
 10  ac_requested            20021 non-null  float64       
 11  total_requested         20020 non-null  object        
 12  Status Comment          20021 non-null  object

In [56]:
df.sample(10)

,Location,PREFIX,project_no,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,fed_requested,ac_requested,total_requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,ftip_no,project_location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO,compare_id_locode
5425,Obligated,HSIPL,5952(184),Ventura County,2016-10-24,2016-10-24,2016-10-26,2016-11-07,2016-11-18,290000.00,0.00,290000.00,Authorized,5952,7,E-76 approved on Prepared on 10/6/2015,0.00,14.00,11.00,SCAG015,Rose Ave. At Simon Way Intersection,Construction Of Traffic Signals At The Interse...,1,NaT,NaT,SCAG,5952,184,True
1854,Obligated,BPMPL,5951(139),Santa Barbara County,2018-04-13,2018-04-13,2018-04-16,2018-04-16,2018-04-20,137221.00,0.00,155000.00,Authorized,5951,5,E-76 approved on,15.00,3.00,4.00,CT14,Sb County Bridge #51c0163 Only,Bridge Preventive Maintenance,3,2018-03-29,2018-04-13,SBCG,5951,139,True
12669,Obligated,RPSTPL,5905(073),Trinity County,2014-02-19,2014-02-19,2014-02-19,2014-02-19,2014-02-25,0.00,0.00,0.00,Authorized,5905,2,E-76 approved on E-76 approved on,0.00,0.00,6.00,NaN,In Lewiston On Lewiston Rd Near Grade School.,Bike/ped Class I & Ii Bikeway (tc),4,NaT,NaT,NON-MPO,5905,073,True
4709,Obligated,HSIPL,5083(014),Redlands,2017-03-15,2017-03-15,2017-03-15,2017-03-28,2017-04-03,-30777.02,0.00,-26553.10,Authorized,5083,8,E-76 approved on Prepared on 2/14/2017,0.00,13.00,6.00,SCAG015,Various Locations Throughout The City Of Redlands,"Upgrade Crosswalks, Install Advance Warning Li...",4,NaT,NaT,SCAG,5083,014,True
2054,Obligated,ER,32L0(047),Inyo County,2018-01-23,2018-03-12,2018-03-16,2018-03-16,2018-04-04,0.00,0.00,0.00,Authorized,5948,9,E-76 approved on,10.00,4.00,19.00,NaN,"Horseshoe Meadow Road,approximately 11 Miles W...",Emergency Opening And Permanent Restoration Of...,2,2018-03-02,2018-03-09,NON-MPO,32L0,047,False
7952,Obligated,STPCML,5916(101),Yuba County,2015-11-30,2015-11-30,2015-11-30,2015-11-30,2015-12-01,-1.01,0.00,193030.21,Authorized,5916,3,E-76 approved on Prepared on 10/6/2015,0.00,0.00,1.00,YUB16028,In Olivehurst At Powerline Rd From 9th Ave To ...,Bike And Pedestrian Improvement,3,NaT,NaT,SACOG,5916,101,True
19313,Obligated,STPL,5126(017),Pinole,2021-06-08,2021-06-14,2021-06-16,2021-06-22,2021-06-24,586000.00,0.00,1155280.50,Authorized,5126,4,E-76 approved on,13.00,8.00,2.00,CC170048,On San Pablo Avenue From City Limits To Pinole...,Rehabilitate Roadway,1,2021-06-01,2021-06-08,MTC,5126,017,True
14651,Obligated,ATPL,5132(047),Fairfield,2019-08-13,2019-09-03,2019-09-03,2019-09-04,2019-09-06,88000.00,0.00,88000,Authorized,5132,4,E-76 approved on,33.00,1.00,2.00,SOL170006,East Tabor Ave. From Railroad Tracks To Olive ...,Install Sidewalk And Widen Existing Sidewalks;...,1,2019-08-01,2019-08-29,MTC,5132,047,True
14723,Obligated,ACST-ER,32L0(207),Santa Cruz County,2019-09-13,2019-09-14,2019-09-23,2019-09-23,2019-09-24,0.00,67433.00,76171,Authorized,5936,5,E-76 approved on,5.00,9.00,1.00,NaN,In The County Of Santa Cruz On Redwood Lodge R...,Reconstruct Slope,3,2019-09-09,2019-09-13,AMBAG,32L0,207,False
1190,Obligated,ER,20A0(039),Humboldt County,2018-06-18,2018-06-18,2018-06-18,2018-06-18,2018-06-28,0.00,0.00,0.00,Authorized,5904,1,E-76 approved on,NaN,0.00,10.00,NaN,"Mattole Road, Pm 15.50",Storm Damage Restoration,3,NaT,NaT,NON-MPO,20A0,039,False


In [57]:
from siuba.data import cars_sql
from siuba import group_by, mutate, show_query

In [58]:
df[_.compare_id_locode == False]

,Location,PREFIX,project_no,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,fed_requested,ac_requested,total_requested,Status Comment,Locode,DIST,Status,dist processing Days,HQ processing Days,FHWA processing Days,ftip_no,project_location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,ProjectID,ProjectNO,compare_id_locode
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,32D0,008,False
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,4820,004,False
21,Obligated,ACSTP,32L0(510),Butte County,2018-12-17,2018-12-20,2018-12-20,2018-12-20,2018-12-21,0.00,2986207.00,3367920.00,Authorized,5912,3,E-76 approved on,43.00,0.00,1.00,NaN,Oroville Quincy Hwy: Location #1 100 Yards Ab...,Emergency Opening And Restoration Of Roadway S...,1,2018-11-07,2018-11-07,BCAG,32L0,510,False
24,Obligated,ER,26E0(001),Napa County,2018-12-18,2018-12-18,2018-12-20,2018-12-20,2018-12-21,941474.00,0.00,1026469.00,Authorized,5921,4,E-76 approved on,13.00,2.00,1.00,NaN,Daf 1: Oak Knoll Ave (mpm 0.20); Daf 2: Oak Kn...,Napa County Earthquake Repair - Pavement Resto...,4,2018-11-30,2018-12-05,MTC,26.00,001,False
54,Obligated,ER,29P0(001),Carlsbad,2018-12-13,2018-12-13,2018-12-13,2018-12-14,2018-12-18,-61058.59,0.00,-53619.67,Authorized,5308,11,E-76 approved on,NaN,1.00,4.00,NaN,"Carlsbad Boulevard, North Of Las Encinas Creek",Emergency Revetment,2,NaT,NaT,SDAG,29P0,001,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19961,Obligated,ACSTP,40A0(076),Palm Springs,2021-08-10,2021-08-10,2021-08-17,2021-09-27,2021-09-27,0.00,263019.00,283856.00,Authorized,5275,8,E-76 approved on,1.00,48.00,0.00,NaN,Farrell Drive At Tahquitz Creek Wash (tc/mcc G...,"Removal Of Mud, Organic Debris, Pavement, Curb...",1,2021-08-09,2021-08-09,SCAG,40A0,076,False
19962,Obligated,ACSTP,40A0(077),Palm Springs,2021-08-10,2021-08-10,2021-08-17,2021-09-27,2021-09-27,0.00,61835.50,65509.00,Authorized,5275,8,E-76 approved on,1.00,48.00,0.00,NaN,Golf Club Road At Tahquitz Creek Golf Course C...,"Removal Of Mud And Debris, Set Up Traffic Cont...",1,2021-08-09,2021-08-09,SCAG,40A0,077,False
19970,Obligated,BR,NBIL(514),Rancho Mirage,2020-12-07,2021-08-31,2021-09-01,2021-09-03,2021-09-16,442650.00,0.00,500000.00,Authorized,5412,8,E-76 approved on,8.00,3.00,13.00,RIV050201,Frank Sinatra Drive Across The Whitewater Rive...,Replace 4-lane Low Water Crossing With 4-lane ...,4,2021-08-23,2021-08-23,SCAG,NBIL,514,False
20000,Obligated,BR,NBIL(516),Calaveras,2021-08-18,2021-08-18,2021-08-30,2021-09-03,2021-09-14,50000.00,0.00,50000.00,Authorized,5463,10,E-76 approved on,6.00,16.00,11.00,HBPLOCAL,Hogan Dam Road Over Bear Creek (lwc 00l0042),Low Water Crossing Replacement (tc),5,2021-08-12,2021-08-18,SCAG,NBIL,516,False


In [59]:
df['Status Comment'].value_counts()

Authorized        19470
Prog Code Z400      147
Prog Code M400       73
Prog Code Z230       60
Prog Code Z003       40
Prog Code M40E       33
Prog Code M0E3       22
Prog Code M23E       17
Prog Code RPS9       14
Prog Code Z40E       14
Prog Code M230       14
Prog Code Z301       14
Prog Code M240       13
Prog Code L23E       10
Prog Code L40E        9
Prog Code Z240        9
Prog Code M003        8
Prog Code L230        6
Prog Code Z23E        6
prog code Z230        3
Prog Code L240        3
Prog Code L400        3
Prog Code L23R        3
Prog Code Z24E        3
Prog Code Q230        2
prog code Z240        2
Prog Code H230        2
Prog Code 33C0        2
Prog Code Z0E3        2
Prog Code M950        2
Prog Code Z300        2
Prog Code M301        2
Prog Code Z950        1
Prog Code M95E        1
Prog Code L24E        1
Prog Code RPS0        1
Prog Code M3E1        1
prog code Z24E        1
prog code             1
prog code Z400        1
Prog Code M30E        1
Prog Code Z233  

In [60]:
df['Location'].value_counts()

Obligated          19470
FTA Transferred      543
DISTRICT               5
HQ                     3
Name: Location, dtype: int64

In [61]:
df['Type of Work'].nunique()

6768

In [62]:
df['PREFIX'].nunique()

388

In [63]:
prefix = df['PREFIX'].value_counts()

In [64]:
prefix.head()

CML      3290
HSIPL    3245
STPL     2763
ER       1568
BRLO     1390
Name: PREFIX, dtype: int64

In [65]:
print((prefix == 1).sum())

113


In [66]:
prefix.sort_index(inplace=True)
prefix

6140000      2
ACNHPI       2
ACST-ER    233
ACSTP      303
ASCTP        1
          ... 
VPPL        19
VPPP         3
VPPPL        5
VPPTCS       1
WZDXL20      1
Name: PREFIX, Length: 388, dtype: int64

In [67]:
#Getting a CSV to create a detailed crosswalk with defintions. 
prefix.to_csv('count_grant_types.csv')

## Convert to CSV for future analysis 

In [68]:
df.to_csv('clean.csv')